# Multitask-Training 
This file is for training the semisupervised multitask model.
The steps performed in the model are :
1. Import the saved model from previous steps
2. Load weights (optional normaly the .h5 file contains the weights of the models)
3. Import the spyros preprocessed numpy images for training and testing with labels
4. Train the multitask network

PS. This file also contains a sample code for zhou heatmaps. ( For HR-CAMs refer the HR-CAMs_Multitask.ipynb notebook)


In [ ]:
# Importing the required libraries

import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras

from sklearn.preprocessing import LabelEncoder


from tensorflow.keras import backend as K
import nibabel as nib
import cv2
import time
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from tensorflow.keras.utils import to_categorical

from prep_data import get_roi, get_all_subjects, get_subject, get_subject_list, pad_to_shape, remove_padding
from PatchGenerator import PatchGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping, TensorBoard
from tensorflow.keras.applications.resnet50 import preprocess_input
import models

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50,DenseNet169,InceptionResNetV2,VGG16
from tensorflow.keras.layers import Conv2DTranspose

from tensorflow.keras.layers import Add, MaxPool2D, Flatten,concatenate,Input,Activation, GlobalAveragePooling2D,GlobalMaxPooling2D, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, Lambda, AveragePooling2D

In [ ]:
# For importing single modalitiy images

"""x_train = np.load('../Data/X_train_ML_3.npy')
y_train = np.load('../Data/Y_train_ML_3.npy')
x_test = np.load('../Data/X_test_ML_3.npy')
y_test = np.load('../Data/Y_test_ML_3.npy')


print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)"""

In [ ]:
"""bkp_X_train = np.copy(x_train)
bkp_Y_train = np.copy(y_train)
bkp_X_test = np.copy(x_test)
bkp_Y_test = np.copy(y_test)"""

In [ ]:
"""
# Coverting single mod to 3 channel by stacking 
# To be used only while importing single mod images
def grayscale_to_3channel(x):
    x = np.squeeze(x)
    return np.stack([x,x,x],axis=-1)

X_train = grayscale_to_3channel(bkp_X_train)
X_test = grayscale_to_3channel(bkp_X_test)

input_shape = X_train.shape[1:]
print(input_shape)
"""

In [ ]:
"""del(bkp_X_test)
del(bkp_X_train)
del(bkp_Y_test)
del(bkp_Y_train)"""

In [ ]:
# Importing the stacked 3 modality images 

x_train_stack = np.load('../Data/X_train_ML_3mod.npy')
y_train_stack = np.load('../Data/Y_train_ML_3mod.npy')
x_test_stack = np.load('../Data/X_test_ML_3mod.npy')
y_test_stack = np.load('../Data/Y_test_ML_3mod.npy')


In [ ]:
plt.imshow(x_test_stack[6,:], cmap='gray')
plt.grid(None)

In [ ]:
# Data gen during runtime
datagen = ImageDataGenerator(
    width_shift_range = [0.2, 0.3],
    height_shift_range = [0.2, 0.3],    
    vertical_flip = True,
    horizontal_flip = True,
    rotation_range=15,
    zoom_range=[0.5,1.0],
    #channel_shift_range=0.2
#     fill_mode = 'constant',
#     cval=0,
)

In [ ]:
# This function is required as we are using multitask images to unwrap the Y labels array and map it with each output
# This function is called during the training of the multitask model
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(4)])

In [ ]:
# Setting batches and creating train and test data gen

batch_size = 8

train_datagen = datagen.flow(X_train,y_train,
                             batch_size=batch_size,
                             shuffle=True,                             
                            )


test_datagen = ImageDataGenerator().flow(X_test,y_test,batch_size=batch_size,
                                         shuffle=True)

#test_datagen = datagen.flow(X_test,y_test,batch_size=batch_size,
#                                         shuffle=True)

print(len(train_datagen),len(test_datagen))

In [ ]:
batch = test_datagen.next()
subject = np.random.randint(0,len(batch[1]))
print(subject,batch[1][subject])

plt.figure()
#plt.subplot(1,2,1)
plt.imshow(batch[0][subject,:,:,0],cmap='gray')

print(np.mean(batch[0]),np.std(batch[0]))

In [ ]:
test_datagen.reset()

In [ ]:
# importing the model consisting of the trained autoencoder's encoder and multitask model
new_model = keras.models.load_model('../Model/TCGA_miccai_1.h5')

In [ ]:
# Loading weights just to be sure
filepath_init = 'weights/TCGA_miccai_1.hdf5'
new_model.load_weights(filepath_init)

In [ ]:
new_model.summary()

In [ ]:
# modifing trainability of the layers
for layer in new_model.layers:
    layer.trainable=False

In [ ]:
# plotting model graph
from tensorflow.keras.utils import plot_model
plot_model(new_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Creating custom loss function to mask the unavailable labels
mask_value = -1
def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
    return K.binary_crossentropy(y_true * mask, y_pred * mask)

#model.compile(loss=masked_loss_function, optimizer='adam', metrics=['accuracy'])

In [ ]:
# setting optimiser and loss_functions
adam = keras.optimizers.Adam(learning_rate=0.00003)

new_model.compile(adam,[masked_loss_function, masked_loss_function, masked_loss_function, masked_loss_function],metrics=['accuracy'])

In [ ]:
train_steps = len(train_datagen)
test_steps = len(test_datagen)
print(train_steps, test_steps)

In [ ]:
filename=os.path.join('logs','TCGA_MT_Miccai_3.csv')
filepath=os.path.join('weights','TCGA_MT_Miccai_3.hdf5')
csv_log = CSVLogger(filename, separator=',', append=True)
checkpoint = ModelCheckpoint(filepath, monitor='val_output1_acc', verbose=1, save_best_only=True)
rl = ReduceLROnPlateau(monitor='val_output1_acc',patience=5,min_delta=0.001,cooldown=5,factor=0.1)
tb = TensorBoard('./logs',histogram_freq=0)
callbacks_list = [csv_log,
                  checkpoint,
                  rl,
                  tb
                 ]

In [ ]:
#filepath1 = os.path.join('weights','TCGA_MT_Miccai_1.hdf5')
if os.path.exists(filepath):
    new_model.load_weights(filepath, by_name=True)

In [ ]:
epochs = 100
nm_H = new_model.fit(generator_wrapper(train_datagen), steps_per_epoch=train_steps, epochs=epochs,
                    verbose = 1,validation_data=generator_wrapper(test_datagen),validation_steps=test_steps,
                    callbacks=callbacks_list)

In [ ]:
# create a new figure for the accuracies
accuracyNames = ["output1_acc", "output2_acc", "output3_acc", 'output4_acc']
plt.style.use("ggplot")
(fig, ax) = plt.subplots(4, 1, figsize=(8, 8))
# loop over the accuracy names
for (i, l) in enumerate(accuracyNames):
    # plot the loss for both the training and validation data
    ax[i].set_title("Accuracy for {}".format(l))
    ax[i].set_xlabel("Epoch #")
    ax[i].set_ylabel("Accuracy")
    ax[i].plot(np.arange(0, epochs), nm_H.history[l], label=l)
    ax[i].plot(np.arange(0, epochs), nm_H.history["val_" + l],
        label="val_" + l)
    ax[i].legend()
# save the accuracies figure
plt.tight_layout()
plt.show()


In [ ]:
lossNames = ["output1_loss", "output2_loss", "output3_loss", 'output4_loss']
plt.style.use("ggplot")
(fig, ax) = plt.subplots(4, 1, figsize=(8, 8))
# loop over the loss names
for (i, l) in enumerate(lossNames):
    # plot the loss for both the training and validation data
    ax[i].set_title("loss for {}".format(l))
    ax[i].set_xlabel("Epoch #")
    ax[i].set_ylabel("loss")
    ax[i].plot(np.arange(0, epochs), nm_H.history[l], label=l)
    ax[i].plot(np.arange(0, epochs), nm_H.history["val_" + l],
        label="val_" + l)
    ax[i].legend()
# save the accuracies figure
plt.tight_layout()
plt.show()

In [ ]:
_test_data, _test_labels = test_datagen.__getitem__(np.random.randint(0,len(test_datagen)))

In [ ]:
## evaluate the model and predict on testing data
print('Evaluate')
a=new_model.evaluate(generator_wrapper(test_datagen),batch_size=batch_size,verbose=1,steps = 1)
print('val_loss , val_acc: ', a)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (accuracy_score, classification_report,
                              confusion_matrix, roc_auc_score, roc_curve)

def get_roc(y_true, y_pred, positive_class_index=0):
    y_pred = np.copy(y_pred)[:, positive_class_index]
    return {'auroc': roc_auc_score(y_true, y_pred), 'roc': roc_curve(y_true, y_pred)}

In [ ]:
y_pred_proba = new_model.predict_generator(test_datagen,verbose=1) 
y_true = y_test
y_pred = y_pred_proba

In [ ]:
y_pred = np.array(y_pred).reshape(-1)

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_true = y_train

In [ ]:
y_pred = np.array(y_pred).reshape(171,4)

In [ ]:
# Classification report for Grade
rint(classification_report(y_true[:,0],y_pred[:,0].round()))

In [ ]:
# Classification report for IDH
print(classification_report(y_true[:,1],y_pred[:,1].round()))

In [ ]:
# Classification report for MGMT
print(classification_report(y_true[:,2],y_pred[:,2].round()))

In [ ]:
# Classification report for 1p19q
print(classification_report(y_true[:,3],y_pred[:,3].round()))

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
n_classes = 4 # number of classes for which roc needs to be plotted
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true[:,i],y_pred[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    #plt.plot([0, 1], [0, 1], 'k--')
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
new_model.save('../Model/TCGA_miccai_trained_2.h5')

new_model.save_weights('weights/TCGA_miccai_trained_2.hdf5')

In [ ]:
print(len(y_test))

In [ ]:
print(y_test)

In [ ]:
#IDH
activation_layer = new_model.get_layer('fireop2_0/relu_expand3x3')
model_n = Model(inputs=new_model.inputs, outputs=activation_layer.output)
    
    
final_dense = new_model.get_layer('output2')
W = final_dense.get_weights()[0]
    
    
    

In [ ]:
img = np.copy(X_test)[[subject_id],:] #4dim img for pred
label_print = np.copy(y_test)[[subject_id],:]
fmaps = model_n.predict(img)[0]

probs = new_model.predict(img)
pred = np.argmax(probs[0])

#get the weights for the relevant class
w = W[:,pred]
print(w.shape)

cam = fmaps.dot(w)
print(cam.shape)

In [ ]:
#upsample to img original size then plot
cam = sp.ndimage.zoom(cam, (16,16), order=1)
cam.shape

In [ ]:
print(label_print)
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
#plt.imshow(img[0][:,:,0], alpha=0.9, cmap='gray')
plt.imshow(cam, cmap ='jet' , alpha=0.65)
plt.grid(b=None)
plt.subplot(1,2,2)
plt.imshow(img[0][:,:,1],cmap='gray')
plt.grid(b=None)
plt.show()